In [ ]:
import tensorflow as tf
import time
import tensorflow_hub as hub
import cv2
from PIL import Image
import uuid

from EfficientDet import *
import os
from glob import glob

# 특정 GPU만 사용

In [ ]:
# gpus = tf.config.experimental.list_physical_devices("GPU")
# if gpus:
# 	# 텐서플로가 세번째 GPU만 사용하도록 제한
# 		try:
# 				tf.config.experimental.set_visible_devices(gpus[-1], "GPU")
# 		except :
# 				print(e)

# pretrained model 가져오기

In [ ]:
model = get_detector("https://tfhub.dev/tensorflow/efficientdet/d4/1")

In [ ]:
dataset_path = "./original_train/"
output_path = "./detected_data/detected_from_train/"
dataset_list = os.listdir("./original_train")

# object detection > 탐지된 객체 저장

In [ ]:
tf.random.set_seed(777)
detected_train_formats = object_detection(model, dataset_list, dataset_path, output_path)

In [ ]:
import pickle


In [ ]:
# with open("detected_train_formats.p", "wb") as out:
#     pickle.dump(detected_train_formats, out)
    

In [ ]:

with open("./detected_train_formats.p","rb") as f:
    detected_train_formats = pickle.load(f)

from glob import glob
image_path = [format["objectImagePath"] for format in detected_train_formats]
detected_images_train = [Image.open(path) for path in image_path]

In [ ]:
from ImageRetrievalClass import ImageRetrievalClass

retrievalInstance = ImageRetrievalClass("IncepResNet", True, False)

In [ ]:
retrievalInstance.readTrainSet(detected_images_train)

In [ ]:
retrievalInstance.buildModel()

In [ ]:
X_train = retrievalInstance.trainTransform()

In [ ]:
np.save("./X_train.npy",X_train)

In [ ]:
E_train = retrievalInstance.predictTrain(X_train) # 연결이 끊기네... (256, 256)으로 바꾸니 해결됨

# 객체 각각의 npy 파일 저장

In [ ]:
for idx, format in enumerate(detected_train_formats):
    filename = format["objectImagePath"][36:-4]
    npyFilename = "./npyFiles/"+filename+".npy"
    np.save(npyFilename, E_train[idx])


# JSON 파일 저장

In [ ]:
import json
for idx, format in enumerate(detected_train_formats):
    npyFilename = "./npyFiles/"+format["objectImagePath"][36:-4]+".npy"
    jsonDict = {
        "objectID":format["objectID"],
        "location":format["location"],
        "tag":format["tag"],
        "objectImagePath" : format["objectImagePath"],
        "npyPath": npyFilename,
        "IMG_URL" : format["IMG_URL"]
    }

    filename = "./jsonFiles/"+format["objectImagePath"][36:-4]+".json"

    with open(filename, "w") as outFile:
        json.dump(jsonDict, outFile, indent=2)